## Методы линеной и логистической регрессий

### Лабораторная работа №3

#### Задание 1
Провести классификацию найденного датасета, методами линеной и логистической регрессий . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

Гиперпараметры:
   - C: обратная сила регуляризации. Меньшие значения C указывают на более сильную регуляризацию.
   - penalty: задает тип регуляризации (например, L1 или L2).
   - solver: алгоритм, используемый для оптимизации весов (например, 'liblinear', 'saga', 'lbfgs' и другие).

In [3]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Загрузка данных
file_path = "Electric_Vehicle_Population_Data.csv"
df = pd.read_csv(file_path)
df.dropna(inplace=True)
# Преобразование категориальной переменной в числовой формат
label_encoder = LabelEncoder()
df['Electric Vehicle Type'] = label_encoder.fit_transform(df['Electric Vehicle Type'])
df = df.sample(n=10000)

# Разделение на признаки (X) и целевую переменную (y)
X = df[['Postal Code', 'Model Year']]
y = df['Electric Vehicle Type']
# Разделение данных на тренировочный и тестовый наборы
# random_state=42 - гарантирует, что данные каждый раз будут одинакого разбиваться
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование признаков (нормализация)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
linear_model = LinearRegression()
linear_model.fit(X_train_scaled, y_train)
linear_accuracy = linear_model.score(X_test_scaled, y_test)
param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l2'],
    'solver': ['liblinear', 'saga', 'lbfgs']
}
logistic = LogisticRegression(max_iter=1000)
grid = GridSearchCV(logistic, param_grid, refit=True, verbose=0)
grid.fit(X_train_scaled, y_train)
# Оценка производительности модели логистической регрессии на тестовом наборе
best_logistic = grid.best_estimator_
logistic_accuracy = best_logistic.score(X_test_scaled, y_test)
print(f"Лучшие гиперпараметры логистической регрессии: {grid.best_params_}")
print(f"Точность модели логистической регрессии: {logistic_accuracy}")
print(f"Точность модели линейной регрессии: {linear_accuracy}")

# Предсказания на тестовой выборке с использованием лучшей модели
predictions = best_logistic.predict(X_test_scaled)
# Отчет по классификации
print("\nОтчет по классификации:")
print(classification_report(y_test, predictions))

Лучшие гиперпараметры логистической регрессии: {'C': 0.1, 'penalty': 'l2', 'solver': 'saga'}
Точность модели логистической регрессии: 0.789
Точность модели линейной регрессии: 0.034227710045715454

Отчет по классификации:
              precision    recall  f1-score   support

           0       0.79      0.99      0.88      1584
           1       0.39      0.03      0.05       416

    accuracy                           0.79      2000
   macro avg       0.59      0.51      0.47      2000
weighted avg       0.71      0.79      0.71      2000

